### 项目目的
目标：根据用户的历史行为记录，预测用户在接下来一天对商品子集的购买行为预测结果

评估指标：Recall和F1

### 数据加载

In [3]:
import pandas as pd
train_user = pd.read_csv('../../dataset/mobile_recommend/tianchi_mobile_recommend_train_user.csv')
train_item = pd.read_csv('../../dataset/mobile_recommend/tianchi_mobile_recommend_train_item.csv')

### 数据查看

In [4]:
train_user.describe()

,user_id,item_id,behavior_type,item_category
count,1.225691e+07,1.225691e+07,1.225691e+07,1.225691e+07
mean,7.170732e+07,2.023084e+08,1.105271e+00,6.846162e+03
std,4.122920e+07,1.167397e+08,4.572662e-01,3.809922e+03
min,4.913000e+03,6.400000e+01,1.000000e+00,2.000000e+00
25%,3.584965e+07,1.014130e+08,1.000000e+00,3.721000e+03
50%,7.292804e+07,2.021359e+08,1.000000e+00,6.209000e+03
75%,1.073774e+08,3.035405e+08,1.000000e+00,1.029000e+04
max,1.424559e+08,4.045625e+08,4.000000e+00,1.408000e+04


In [13]:
train_user.count()

user_id          12256906
item_id          12256906
behavior_type    12256906
user_geohash      3922082
item_category    12256906
time             12256906
dtype: int64

In [14]:
train_user.columns

Index(['user_id', 'item_id', 'behavior_type', 'user_geohash', 'item_category',
       'time'],
      dtype='object')

In [15]:
train_user.head(10)

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21
5,93784494,337869048,1,NaN,3979,2014-12-03 20
6,94832743,105749725,1,NaN,9559,2014-12-13 20
7,95290487,76866650,1,NaN,10875,2014-11-27 16
8,96610296,161166643,1,NaN,3064,2014-12-11 23
9,100684618,21751142,3,NaN,2158,2014-12-05 23


In [17]:
train_user.user_geohash.describe()

count     3922082
unique     575458
top       94ek6ke
freq         1052
Name: user_geohash, dtype: object

### 从用户训练集表中，可以得到以下信息：
1.存在6个字段，分别为'user_id', 'item_id', 'behavior_type', 'user_geohash', 'item_category','time'，对应用户id、商品id、行为类型、用户地理位置、商品目录、时间

2.用户行为分为4类，值分别为1、2、3、4,对应浏览、收藏、加购物车、购买

3.地理位置缺失值较多,应如何处理


In [5]:
train_item.describe()

,item_id,item_category
count,4.807230e+05,480723.000000
mean,1.983196e+08,6790.729437
std,1.180976e+08,3404.598754
min,2.651000e+03,2.000000
25%,9.032219e+07,4533.000000
50%,2.088348e+08,6648.000000
75%,2.956942e+08,9933.000000
max,4.045624e+08,14071.000000


In [6]:
train_item.columns

Index(['item_id', 'item_geohash', 'item_category'], dtype='object')

In [7]:
train_item.head()

,item_id,item_geohash,item_category
0,312051294,NaN,8270
1,99999754,NaN,7393
2,131746128,NaN,7876
3,385731330,NaN,10544
4,100004415,NaN,3064


In [49]:
# 将文本格式的时间转换为时间戳格式，方便计算
train_user.time = pd.to_datetime(train_user.time)

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02:00:00
1,97726136,383583590,1,NaN,5894,2014-12-09 20:00:00
2,98607707,64749712,1,NaN,2883,2014-12-18 11:00:00
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10:00:00
4,98145908,290208520,1,NaN,13926,2014-12-16 21:00:00


### 数据探索

In [66]:
behavior_type_dict = {1:'browse', 2:'collection',3:'cart',4:'buy'}
behavior_type_transform = train_user.behavior_type.apply(lambda x:behavior_type_dict.get(x))

In [78]:
behavior_type_transform.describe()

count     12256906
unique           4
top         browse
freq      11550581
Name: behavior_type, dtype: object

In [79]:
train_user['behavior_type_transform'] = behavior_type_transform

In [80]:
train_user.head(10)

,user_id,item_id,behavior_type,user_geohash,item_category,time,behavior_type_transform
0,98047837,232431562,1,NaN,4245,2014-12-06 02:00:00,browse
1,97726136,383583590,1,NaN,5894,2014-12-09 20:00:00,browse
2,98607707,64749712,1,NaN,2883,2014-12-18 11:00:00,browse
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10:00:00,browse
4,98145908,290208520,1,NaN,13926,2014-12-16 21:00:00,browse
5,93784494,337869048,1,NaN,3979,2014-12-03 20:00:00,browse
6,94832743,105749725,1,NaN,9559,2014-12-13 20:00:00,browse
7,95290487,76866650,1,NaN,10875,2014-11-27 16:00:00,browse
8,96610296,161166643,1,NaN,3064,2014-12-11 23:00:00,browse
9,100684618,21751142,3,NaN,2158,2014-12-05 23:00:00,cart


In [98]:
train_user['browse'] = train_user.behavior_type.apply(lambda x: 1 if x==1 else 0)
train_user['collection'] = train_user.behavior_type.apply(lambda x: 1 if x==2 else 0)
train_user['cart'] = train_user.behavior_type.apply(lambda x: 1 if x==3 else 0)
train_user['buy'] = train_user.behavior_type.apply(lambda x: 1 if x==4 else 0)

In [106]:
train_user_count = train_user.groupby(['user_id', 'item_id'])['browse','collection','cart','buy'].sum().reset_index()

In [107]:
train_user_count.describe()

,user_id,item_id,browse,collection,cart,buy
count,4.686904e+06,4.686904e+06,4.686904e+06,4.686904e+06,4.686904e+06,4.686904e+06
mean,7.156401e+07,2.023695e+08,2.464437e+00,5.175186e-02,7.330297e-02,2.564699e-02
std,4.113416e+07,1.167612e+08,2.327758e+00,2.356270e-01,3.177246e-01,1.984310e-01
min,4.913000e+03,6.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.575874e+07,1.014752e+08,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,7.278901e+07,2.022472e+08,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.072237e+08,3.036372e+08,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.424559e+08,4.045625e+08,1.250000e+02,9.000000e+00,1.400000e+01,3.100000e+01


In [118]:
train_user_count.head()

,user_id,item_id,browse,collection,cart,buy
0,4913,315532,1,0,0,0
1,4913,876969,6,0,0,0
2,4913,2741340,1,0,0,0
3,4913,3007091,3,0,0,0
4,4913,4210607,1,0,0,0


In [111]:
train_user_count.shape

(4686904, 6)

In [113]:
train_user_count_item = train_user.groupby(['item_id'])['browse','collection','cart','buy'].sum().reset_index()

In [114]:
train_user_count_item.describe()

,item_id,browse,collection,cart,buy
count,2.876947e+06,2.876947e+06,2.876947e+06,2.876947e+06,2.876947e+06
mean,2.022864e+08,4.014874e+00,8.431021e-02,1.194196e-01,4.178214e-02
std,1.167983e+08,8.443660e+00,3.448123e-01,4.773012e-01,2.908267e-01
min,6.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.011113e+08,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.022034e+08,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.033948e+08,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.045625e+08,1.431000e+03,2.400000e+01,5.600000e+01,5.000000e+01


In [119]:
train_user_count_item.head()

,item_id,browse,collection,cart,buy
0,64,2,0,0,0
1,496,2,0,0,0
2,528,2,0,0,0
3,579,2,0,0,0
4,581,4,0,0,0


In [120]:
train_user_count_user = train_user.groupby(['user_id'])['browse','collection','cart','buy'].sum().reset_index()

In [122]:
train_user_count_user.describe()

,user_id,browse,collection,cart,buy
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,7.144897e+07,1155.058100,24.255600,34.356400,12.020500
std,4.119949e+07,1430.052774,73.900635,63.889429,19.050621
min,4.913000e+03,1.000000,0.000000,0.000000,0.000000
25%,3.547193e+07,297.000000,0.000000,2.000000,2.000000
50%,7.251422e+07,703.000000,2.000000,12.000000,7.000000
75%,1.073052e+08,1461.000000,18.000000,39.000000,15.000000
max,1.424559e+08,27720.000000,2935.000000,1810.000000,809.000000


In [123]:
train_user_count_user.head()

,user_id,browse,collection,cart,buy
0,4913,1658,49,29,6
1,6118,112,4,0,1
2,7528,183,1,24,6
3,7591,824,0,14,21
4,12645,248,2,10,8
